In [ ]:
from google.colab import drive
import json
import pandas as pd

In [ ]:
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/'

def json_to_df(file_name):
    with open(file_path+file_name, 'r',encoding='utf-8') as f:
        data = json.load(f)

    df = pd.json_normalize(data, record_path = ['sentences'], meta=['id'], record_prefix='sent_')
    df = df.drop(columns = ['sent_id', 'sent_speaker', 'sent_types', 'sent_intensity_sum',
                            'sent_text', 'sent_intensity', 'sent_votes', 'sent_frame_id', 'sent_mapped_slots'])
    df.replace({False:'0', True:'1'}, inplace=True)

    df['sent_origin_text'] = df['sent_origin_text'].str.replace('\\','')
    combined_text = df.groupby('id')['sent_origin_text'].agg('[SEP]'.join)

    combined_label = df.groupby('id')['sent_is_immoral'].agg(' '.join)
    combined_label = combined_label.str.split(' ')
    for i in range(len(combined_label)):
        combined_label[i] = [int(j) for j in combined_label[i]]

    merged_df = pd.concat([combined_text, combined_label], axis = 1)
    merged_df.reset_index(drop = True, inplace=True)
    merged_df.columns = ['text', 'label']

    return merged_df

df = json_to_df('talksets-train-1.json')
df

Mounted at /content/drive


<ipython-input-2-8988c43353de>:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['sent_origin_text'] = df['sent_origin_text'].str.replace('\\','')


,text,label
0,30만이 저 가수를 응원한다더라[SEP]나는 쟤가 혐오스러워서 안 나왔으면 ㅋㅋㅋ[...,"[0, 1, 1]"
1,저 프로그램은 30명이 문제가 됐더라[SEP]30명 실명 다 공개하면 될걸 저렇게 ...,"[0, 0, 1]"
2,30분 지나도 댓글 세자리도 안 나오는 프로그램이네[SEP]맞아 그냥 망한 프로그램...,"[0, 1, 0]"
3,30분전만해도 시즌 3 없다고 하던데[SEP]맞아 원래 제작진들이 변덕 심한 제레기...,"[0, 1, 1]"
4,30살 아재는 20살 만나도 돼?[SEP]당연히 안되지 쓰레기 같은 소리하네[SEP...,"[0, 1, 0]"
...,...,...
19684,"아이고,, 꼴에 박사니 뭐니 지 주제도 모르고 실컷 떠들다가 인생망한것같아보이네[S...","[1, 1]"
19685,혹시 머리에 문제있는거아니야? 허언증 말기 같은데 검사한번 받아보는게..[SEP]왜...,"[1, 0]"
19686,그렇게 일방적으로 니 입장만 고수할거면 난 할 말 없어.[SEP]괜히 찔리니까 입 ...,"[0, 1]"
19687,한의원이 진짜 의학적으로 효과가 있긴해? 그냥 사기꾼 같은데[SEP]돈잘버니까 어...,"[1, 1, 1]"


In [ ]:
type(df['label'][0][0])

int

In [ ]:
df.to_csv(file_path+'sequence_merge_data.csv', encoding = 'utf-8-sig', index = False)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df['text'], df['label'], random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state=0)

In [ ]:
train_df = pd.DataFrame({'text': X_train, 'label': y_train})
test_df = pd.DataFrame({'text': X_test, 'label': y_test})
val_df = pd.DataFrame({'text': X_val, 'label': y_val})

print(train_df.shape)
print(test_df.shape)
print(val_df.shape)

(11074, 2)
(3692, 2)
(4923, 2)


In [ ]:
train_df.to_csv(file_path+'sequence_train_data.csv', encoding = 'utf-8-sig', index = False)
val_df.to_csv(file_path+'sequence_validation_data.csv', encoding = 'utf-8-sig', index = False)
test_df.to_csv(file_path+'sequence_test_data.csv', encoding = 'utf-8-sig', index = False)